# You first need to make a copy of this notebook in order to replace your API key in the code. Go to File ---> Save a copy in Drive.

In [2]:
#@title "Install the dependencies"
!pip3 install salesforce-lavis
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Please change YOUR API KEY to your OpenAI API key.

In [ ]:
#@title
import cv2
import math
import torch
from PIL import Image

from lavis.models import load_model_and_preprocess
from google.colab import files
import os
import openai
openai.api_key = "YOUR API KEY"
## Configure the parameters for GPT-3"
model_engine = "text-davinci-003"
max_length = 50
top_p = 0
n_best = 1
temperature = 0
batch_size = 16 ##### ADJUST ACCORDING TO HARDWARE!

# Define the input directory path
input_dir = "/content/input"

# Define the output directory path
output_dir = "/content/output"

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)

# Allow the user to upload multiple files
uploaded = files.upload()
for name, data in uploaded.items():
    with open(os.path.join(input_dir, name), "wb") as f:
        f.write(data)

# Get the list of video files in the input directory
video_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".mp4")]


for video_file in video_files:
    # Get the name of the video file
    video_filename = os.path.splitext(os.path.basename(video_file))[0]

    # Define the output text file path
    output_file = os.path.join(output_dir, "predictions_" + video_filename + ".txt")

    # Print the video file name and output file
    print("Archivo cargado:", video_filename)
    print("Archivo de salida:", output_file)

    # Load the video file
    cap = cv2.VideoCapture(video_file)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create a list with the frames to be extracted
    frames_to_extract = [5, math.ceil(total_frames * 0.25), math.ceil(total_frames * 0.5), math.ceil(total_frames * 0.9), math.ceil(total_frames * 0.99) - 2]

    # Iterate through each frame to extract and predict
    with open(output_file, "w") as f:
        for frame_number in frames_to_extract:
            # set the current frame and read it
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
            ret, frame = cap.read()

            
            if not ret:
                break

            # load blip model
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model, vis_processors, _ = load_model_and_preprocess(
                name="blip_caption", model_type="large_coco", is_eval=True, device=device
            )

            # Preprocess the frame for the model
            preprocessed_frame = vis_processors["eval"](Image.fromarray(frame)).unsqueeze(0).to(device)

            # Generate the model prediction on the frame
            model_response = model.generate({"image": preprocessed_frame})[0]

            # Write the prediction to the output file
            f.write(f"Frame {frame_number}: {model_response}\n")

      # Read the predictions from the output file
    with open(output_file, 'r') as file:
        text = file.read()
    
    # Add the video name to the OpenAI prompt
    prompt = text + f"Video name is {video_file}. Video short description:"
    print(prompt)
    
    # Send the prompt to GPT-3 to complete the action
    response = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=max_length,
        n=1,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        best_of=n_best
    )
    
    # Get the response from GPT-3
    action = response.choices[0].text.strip()
    
    # Print the action suggested by GPT-3
    print(action)
    
    # Add the action to the last line of the output file
    with open(output_file, 'a') as file:
        file.write(f"{action}\n")

# Close the video file
    cap.release()

print("Processing completed.")





In [ ]:
#@title zip and download files
!zip -r output.zip /content/output
from google.colab import video_files
files.download('/content/output.zip')

In [ ]:
import csv
import os

txt_folder = '/content/output'
csv_file = '/content/output_data.csv'
data = []

for file_name in os.listdir(txt_folder):
    if file_name.endswith('.txt'):
      with open(os.path.join(txt_folder, file_name), 'r') as f:
                lines = f.readlines()
                name = file_name
                second_line = lines[1].strip()
                last_line = lines[-1].rstrip() if lines[-1].rstrip() else lines [-2].rstrip()
                data.append([name, second_line, last_line])

with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)

from google.colab import files
files.download(csv_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!rm -R /content/*